In [15]:
import findspark
findspark.init()

In [16]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [17]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = ' pyspark-shell'

In [18]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName("Titanic - Analytics - MLlib")
    .config("spark.sql.warehouse.dir","hdfs://localhost:9000/warehouse")
    .enableHiveSupport()
    .getOrCreate())

In [19]:
titanic_raw = (spark.read
                    .option("inferSchema", "true")
                    .option('header', 'true')
                    .csv("hdfs://localhost:9000/datalake/raw/kaggle/titanic/")
                    .cache())

In [20]:
titanic_raw.limit(5).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [21]:
titanic_raw.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [22]:
passengers_count = titanic_raw.count()
print (f"Total number of passenger: {passengers_count}")

Total number of passenger: 891


In [23]:
titanic_raw.summary().toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,25%,223,0,2,None,None,20.0,0,0,19996.0,7.8958,None,None
5,50%,446,0,3,None,None,28.0,0,0,236171.0,14.4542,None,None
6,75%,669,1,3,None,None,38.0,1,0,347743.0,31.0,None,None
7,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [24]:
titanic_raw.groupBy("Survived").count().toPandas()

,Survived,count
0,1,342
1,0,549


In [25]:
titanic_raw.groupBy("Sex","Survived").count().toPandas()

,Sex,Survived,count
0,male,0,468
1,female,1,233
2,female,0,81
3,male,1,109


In [26]:
titanic_raw.groupBy("Pclass","Survived").count().toPandas()

,Pclass,Survived,count
0,1,0,80
1,3,1,119
2,1,1,136
3,2,1,87
4,2,0,97
5,3,0,372


In [27]:
titanic_df = titanic_raw

In [28]:
from pyspark.sql.functions import isnull, when, count, col

# Option 1
titanic_df.select([count(when(isnull(c), c)).alias(c) for c in titanic_df.columns]).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,0,0,0,177,0,0,0,0,687,2


In [29]:
# Option 2
titanic_df.summary("count").toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889


In [30]:
titanic_df = titanic_df.drop("link")

In [31]:
titanic_df.groupBy("Embarked").count().toPandas()

,Embarked,count
0,Q,77
1,None,2
2,C,168
3,S,644


In [32]:
titanic_df = titanic_df.na.fill({"Embarked" : 'S'})

In [33]:
from pyspark.sql.functions import regexp_extract
titanic_df = titanic_df.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

In [34]:
titanic_df.limit(5).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Initial
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,Mr


In [35]:
titanic_df.select("Initial").distinct().sort("Initial").toPandas()

,Initial
0,Capt
1,Col
2,Countess
3,Don
4,Dr
5,Jonkheer
6,Lady
7,Major
8,Master
9,Miss


In [36]:
titanic_df = titanic_df.replace(
               ['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Mr',  'Mr','Mr','Mr','Mr','Mr'])

In [37]:
titanic_df.select("Initial").distinct().toPandas()

,Initial
0,Miss
1,Master
2,Mr
3,Mrs


In [38]:
from pyspark.sql.functions import round

avg_age_df = (titanic_df.groupby('Initial').avg('Age')
                        .withColumnRenamed("avg(Age)","Age"))
avg_age_df.toPandas()

,Initial,Age
0,Miss,21.860000
1,Master,4.574167
2,Mr,33.022727
3,Mrs,35.981818


In [39]:
titanic_df_noage = titanic_df.where(col("Age").isNull()).drop("Age")
titanic_df_noage.limit(1).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Initial
0,6,0,3,"Moran, Mr. James",male,0,0,330877,8.4583,None,Q,Mr


In [40]:
titanic_df_noage_with_avg = titanic_df_noage.join(avg_age_df, "Initial")
titanic_df_noage_with_avg.limit(1).toPandas()

,Initial,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age
0,Mr,6,0,3,"Moran, Mr. James",male,0,0,330877,8.4583,None,Q,33.022727


In [41]:
titanic_df_fixed = (titanic_df.where(col("Age").isNotNull())
                      .unionByName(titanic_df_noage_with_avg))

titanic_df_fixed.where(col("Age").isNull()).count()

0

In [42]:
titanic_df = titanic_df_fixed

In [43]:
titanic_df = titanic_df.withColumn("Family_Size",col('SibSp')+col('Parch'))

In [44]:
titanic_df.groupBy("Family_Size").count().toPandas()

,Family_Size,count
0,1,161
1,6,12
2,3,29
3,5,22
4,4,15
5,7,6
6,2,102
7,0,537
8,10,7


In [45]:
from pyspark.sql.functions import lit
titanic_df = titanic_df.withColumn("Alone",when(titanic_df["Family_Size"] == 0, 1).otherwise(lit(0)))

In [46]:
titanic_df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked',
 'Initial',
 'Family_Size',
 'Alone']

In [47]:
from pyspark.sql.functions import countDistinct

titanic_df.select([countDistinct(c).alias(c) for c in titanic_df.columns]).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Initial,Family_Size,Alone
0,891,2,3,891,2,92,7,7,681,248,147,3,4,9,2


In [48]:
titanic_df = titanic_df.drop("PassengerId","Name","Ticket","Initial", "Family_Size")

In [49]:
titanic_df.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- Alone: integer (nullable = false)



In [50]:
titanic_df = titanic_df.select(col('Survived').cast('double'),
                              col('Pclass').cast('double'),
                              col('Sex'),
                              col('Age').cast('double'),
                              col('SibSp').cast('double'),
                              col('Parch').cast('double'),
                              col('Fare').cast('double'),
                              col('Embarked'),
                              col('Alone').cast('double')
                             )

In [51]:
titanic_df.printSchema()

root
 |-- Survived: double (nullable = true)
 |-- Pclass: double (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- Alone: double (nullable = false)



In [52]:
label_column = "Survived"

categoricalCols = [field for (field, dataType) in titanic_df.dtypes if ((dataType == "string") & (field != label_column))]
numericCols = [field for (field, dataType) in titanic_df.dtypes if ((dataType == "double") & (field != label_column))]

print (f"categorical columns: {categoricalCols}")
print (f"numerical columns: {numericCols}")

categorical columns: ['Sex', 'Embarked']
numerical columns: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Alone']


In [53]:
indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

print (f"StringIndexer column names: {indexOutputCols}")
print (f"OHE column names: {oheOutputCols}")

StringIndexer column names: ['SexIndex', 'EmbarkedIndex']
OHE column names: ['SexOHE', 'EmbarkedOHE']


In [54]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid="skip")

oheEncoder = OneHotEncoder(inputCols=indexOutputCols,outputCols=oheOutputCols)

In [55]:
temp_df = stringIndexer.fit(titanic_df).transform(titanic_df)
temp_df.toPandas()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Alone,SexIndex,EmbarkedIndex
0,0.0,3.0,male,22.000000,1.0,0.0,7.2500,S,0.0,0.0,0.0
1,1.0,1.0,female,38.000000,1.0,0.0,71.2833,C,0.0,1.0,1.0
2,1.0,3.0,female,26.000000,0.0,0.0,7.9250,S,1.0,1.0,0.0
3,1.0,1.0,female,35.000000,1.0,0.0,53.1000,S,0.0,1.0,0.0
4,0.0,3.0,male,35.000000,0.0,0.0,8.0500,S,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,3.0,male,33.022727,0.0,0.0,7.2292,C,1.0,0.0,1.0
887,0.0,3.0,female,21.860000,8.0,2.0,69.5500,S,0.0,1.0,0.0
888,0.0,3.0,male,33.022727,0.0,0.0,9.5000,S,1.0,0.0,0.0
889,0.0,3.0,male,33.022727,0.0,0.0,7.8958,S,1.0,0.0,0.0


In [56]:
oheEncoder.fit(temp_df).transform(temp_df).toPandas()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Alone,SexIndex,EmbarkedIndex,SexOHE,EmbarkedOHE
0,0.0,3.0,male,22.000000,1.0,0.0,7.2500,S,0.0,0.0,0.0,(1.0),"(1.0, 0.0)"
1,1.0,1.0,female,38.000000,1.0,0.0,71.2833,C,0.0,1.0,1.0,(0.0),"(0.0, 1.0)"
2,1.0,3.0,female,26.000000,0.0,0.0,7.9250,S,1.0,1.0,0.0,(0.0),"(1.0, 0.0)"
3,1.0,1.0,female,35.000000,1.0,0.0,53.1000,S,0.0,1.0,0.0,(0.0),"(1.0, 0.0)"
4,0.0,3.0,male,35.000000,0.0,0.0,8.0500,S,1.0,0.0,0.0,(1.0),"(1.0, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,3.0,male,33.022727,0.0,0.0,7.2292,C,1.0,0.0,1.0,(1.0),"(0.0, 1.0)"
887,0.0,3.0,female,21.860000,8.0,2.0,69.5500,S,0.0,1.0,0.0,(0.0),"(1.0, 0.0)"
888,0.0,3.0,male,33.022727,0.0,0.0,9.5000,S,1.0,0.0,0.0,(1.0),"(1.0, 0.0)"
889,0.0,3.0,male,33.022727,0.0,0.0,7.8958,S,1.0,0.0,0.0,(1.0),"(1.0, 0.0)"


In [57]:
assemblerInputs = oheOutputCols + numericCols
print("Feature columns: ",assemblerInputs)

Feature columns:  ['SexOHE', 'EmbarkedOHE', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Alone']


In [58]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol="features")

In [59]:
from pyspark.ml import Pipeline

test_pipeline = Pipeline(stages = [stringIndexer, oheEncoder, vecAssembler])
features_df = test_pipeline.fit(titanic_df).transform(titanic_df)
features_df.limit(2).toPandas()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Alone,SexIndex,EmbarkedIndex,SexOHE,EmbarkedOHE,features
0,0.0,3.0,male,22.0,1.0,0.0,7.2500,S,0.0,0.0,0.0,(1.0),"(1.0, 0.0)","[1.0, 1.0, 0.0, 3.0, 22.0, 1.0, 0.0, 7.25, 0.0]"
1,1.0,1.0,female,38.0,1.0,0.0,71.2833,C,0.0,1.0,1.0,(0.0),"(0.0, 1.0)","[0.0, 0.0, 1.0, 1.0, 38.0, 1.0, 0.0, 71.2833, 0.0]"


In [60]:
seed=11

In [61]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import LinearSVC

lr = LogisticRegression(labelCol="Survived", featuresCol="features")
dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="features",seed=seed)
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features",maxDepth=10,seed=seed)
gbt = GBTClassifier(labelCol="Survived", featuresCol="features",maxIter=10,seed=seed)
nb = NaiveBayes(labelCol="Survived", featuresCol="features")
svm = LinearSVC(labelCol="Survived", featuresCol="features")

classifiers = [lr,dt,rf,gbt,nb,svm]
classifiers

[LogisticRegression_442016d83d81,
 DecisionTreeClassifier_f9255fb321e8,
 RandomForestClassifier_c7e5fd4dd5a0,
 GBTClassifier_0c35ac6b2c3a,
 NaiveBayes_1dde2dc93ea1,
 LinearSVC_8fc05e9e20b2]

In [62]:
from pyspark.ml import Pipeline

def create_pipeline(classifier):
    return Pipeline(stages = [stringIndexer, oheEncoder, vecAssembler, classifier])

pipelines = [create_pipeline(classifier) for classifier in classifiers]
pipelines

[Pipeline_3e427d3d0bb4,
 Pipeline_4f7eb7d6a0e4,
 Pipeline_96c2b23ac1a7,
 Pipeline_eee8342afcc6,
 Pipeline_45bbc06f971c,
 Pipeline_3045782b6bda]

In [63]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Survived",  metricName="accuracy")

In [64]:
(trainingData, testData) = titanic_df.randomSplit([0.8,0.2],seed=seed)

In [65]:
(trainingData1, testData1) = titanic_df.where("Survived=0").randomSplit([0.8,0.2],seed=seed)
(trainingData2, testData2) = titanic_df.where("Survived=1").randomSplit([0.8,0.2],seed=seed)

traininData = trainingData1.unionByName(trainingData2)
testData = testData1.unionByName(testData2)

In [66]:
models = [pipeline.fit(trainingData) for pipeline in pipelines]
models

[PipelineModel_96ad9c72ddc9,
 PipelineModel_8ef9d6817a74,
 PipelineModel_80a6f9dcee9f,
 PipelineModel_09b70e625197,
 PipelineModel_667a95a57a4b,
 PipelineModel_1b70b65fc390]

In [67]:
names = []
values = [] 
for model in models:
    prediction_df = model.transform(testData)
    accuracy = evaluator.evaluate(prediction_df)
    names.append(type(model.stages[-1]).__name__) # the algorithm is the last stage in the pipeline
    values.append(accuracy)

data = {'name':names,'accuracy':values,'model':models}
df = pd.DataFrame(data)
df.sort_values(by=['accuracy'], inplace=True, ascending=False)  
df

,name,accuracy,model
2,RandomForestClassificationModel,0.858696,PipelineModel_80a6f9dcee9f
5,LinearSVCModel,0.826087,PipelineModel_1b70b65fc390
0,LogisticRegressionModel,0.804348,PipelineModel_96ad9c72ddc9
1,DecisionTreeClassificationModel,0.804348,PipelineModel_8ef9d6817a74
3,GBTClassificationModel,0.798913,PipelineModel_09b70e625197
4,NaiveBayesModel,0.701087,PipelineModel_667a95a57a4b


In [68]:
best_model=df.iloc[0]['model']

In [69]:
best_model.transform(testData).groupby("Survived").pivot("prediction").count().toPandas()

,Survived,0.0,1.0
0,0.0,100,15
1,1.0,11,58


In [70]:
modelPath = "hdfs://localhost:9000/model-registry/titanic-survival-classifier"
best_model.write().overwrite().save(modelPath)

In [71]:
from pyspark.ml import PipelineModel
savedModel = PipelineModel.load(modelPath)

In [72]:
predictions = savedModel.transform(testData)
predictions.select("features", "Survived", "prediction").limit(200).toPandas()

,features,Survived,prediction
0,"[1.0, 1.0, 0.0, 1.0, 19.0, 3.0, 2.0, 263.0, 0.0]",0.0,0.0
1,"[1.0, 1.0, 0.0, 1.0, 21.0, 0.0, 1.0, 77.2875, 0.0]",0.0,0.0
2,"[1.0, 0.0, 1.0, 1.0, 28.0, 1.0, 0.0, 82.1708, 0.0]",0.0,1.0
3,"[1.0, 1.0, 0.0, 1.0, 29.0, 0.0, 0.0, 30.0, 1.0]",0.0,0.0
4,"[1.0, 1.0, 0.0, 1.0, 29.0, 1.0, 0.0, 66.6, 0.0]",0.0,0.0
...,...,...,...
179,"(0.0, 0.0, 0.0, 3.0, 21.86, 1.0, 0.0, 15.5, 0.0)",1.0,1.0
180,"[0.0, 0.0, 1.0, 3.0, 35.981818181818184, 0.0, 2.0, 22.3583, 0.0]",1.0,1.0
181,"[0.0, 1.0, 0.0, 3.0, 35.981818181818184, 1.0, 0.0, 16.1, 0.0]",1.0,1.0
182,"[1.0, 0.0, 0.0, 3.0, 33.02272727272727, 0.0, 0.0, 7.75, 1.0]",1.0,0.0
